In [1]:
###params
import os
try:
    os.environ['OPT']
except:
    os.environ['OPT']='train'
opt = os.environ['OPT']
opt='test'

In [2]:
try:
    print(f'name of the variable:{os.environ["DB_HOST"]}')
except:
    print('variable not there')

name of the variable:docker.for.mac.host.internal


# Description

- Process the dataset to train the model
- Create the labels

to test the notebook
- papermill notebooks/feature_selection.ipynb notebooks/output/code_exec.ipynb --log-output

Run from a docker image

- to build: docker build . -t feature_selection
- to run: docker run -it  -v /Users/mendes/tech_interview/Kensho/applications/feature_selection/notebooks:/notebooks -e EXECUTION_ID=444444 -e DB_HOST=docker.for.mac.host.internal -e OPT=train feature_selection -p 3306:3306 

obs: Change the DB_HOST env variable when not in MAC

In [3]:
pwd

'/'

# Import libraries and define functions and paths

## libraries

In [4]:
import pandas as pd
import datetime
import os
import sqlalchemy as db
from sklearn.preprocessing import LabelEncoder
import pickle

## functions

In [5]:
def get_connection():
    try:
        os.environ["DB_HOST"]
    except:
        os.environ["DB_HOST"] = "localhost"
        
    print(f'name of the variable inside function:{os.environ["DB_HOST"]}')



    # specify database configurations
    config = {
        'host': os.environ['DB_HOST'],
        'port': 3306,
        'user': 'mendes',
        'password': 'test',
        'database': 'test_db'
    }
    db_user = config.get('user')
    db_pwd = config.get('password')
    db_host = config.get('host')
    db_port = config.get('port')
    db_name = config.get('database')
    # specify connection string
    connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'
    # connect to database
    engine = db.create_engine(connection_str)
    return engine

## Paths

In [6]:
!echo $DB_HOST

docker.for.mac.host.internal


# Connect SQL DB and transfer data

In [7]:
engine = get_connection()

name of the variable inside function:docker.for.mac.host.internal


## Process dataset

In [8]:
with engine.connect() as conn:
    sql = f'SELECT * FROM {opt};'
    data = pd.read_sql(sql,conn)

In [9]:
if opt=='train' or opt=='test':
    ##separate labels
    labels = data['species']

    ## drop information about timestamp
    data = data.drop(['timestamp','species','index'],axis=1)
else:
    #TEMPORARY
    ##separate labels
    labels = data['species']

    ## drop information about timestamp
    data = data.drop(['timestamp','species','index'],axis=1)

In [10]:
path = os.path.abspath(os.getcwd())+'/'



if 'notebooks' not in path:
    path = path+'/notebooks/'
    
print('THIS IS THE:'+path)

if opt =='train':
    ## encode the labels to numerical values
    le = LabelEncoder()
    l = le.fit_transform(labels)
    
    with open(path+'label_encoder.p','wb') as handle:
        pickle.dump(le,handle,protocol=pickle.HIGHEST_PROTOCOL)  
elif opt=='test':
    with open(path+'label_encoder.p','rb') as handle:
        le = pickle.load(handle)
    l=le.transform(labels)
elif opt=='validation':
    pass

THIS IS THE:///notebooks/


In [11]:
path

'///notebooks/'

In [12]:
ls notebooks

feature_selection.ipynb  label_encoder.p  output/


In [13]:
with engine.connect() as conn:
    if opt=='train' or opt=='test':
        df = pd.concat([data,pd.Series(l,name='label')],axis=1)
        df.to_sql(f'{opt}_preprocessed',conn,index=False,if_exists='replace')
    elif opt=='validation':
        df=data
        df.to_sql(f'{opt}_preprocessed',conn,index=False,if_exists='replace')

In [14]:
print(f'worked feature selection using . {opt}')

worked feature selection using . test


In [15]:
df

,sepal_length,sepal_width,petal_length,petal_width,label
